In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Airbnb A/B Testing - Data Cleaning and Simulation\n",
    "\n",
    "This notebook demonstrates data cleaning and A/B test group assignment for Airbnb listings."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy import stats\n",
    "import sqlite3\n",
    "\n",
    "sns.set_style('whitegrid')\n",
    "np.random.seed(42)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Generate Sample Data\n",
    "\n",
    "Generate realistic Airbnb listing data for A/B testing simulation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate sample data\n",
    "n_records = 5000\n",
    "rng = np.random.default_rng(42)\n",
    "\n",
    "room_types = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']\n",
    "neighborhoods = ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']\n",
    "\n",
    "data = {\n",
    "    'listing_id': range(1, n_records + 1),\n",
    "    'name': [f'Property {i}' for i in range(1, n_records + 1)],\n",
    "    'host_id': rng.integers(1000, 9999, n_records),\n",
    "    'neighborhood': rng.choice(neighborhoods, n_records),\n",
    "    'room_type': rng.choice(room_types, n_records, p=[0.5, 0.35, 0.1, 0.05]),\n",
    "    'price': rng.integers(50, 500, n_records),\n",
    "    'minimum_nights': rng.choice([1, 2, 3, 7, 30], n_records, p=[0.4, 0.3, 0.15, 0.1, 0.05]),\n",
    "    'number_of_reviews': rng.integers(0, 200, n_records),\n",
    "    'reviews_per_month': rng.uniform(0, 5, n_records),\n",
    "    'availability_365': rng.integers(0, 365, n_records),\n",
    "    'instant_bookable': rng.choice([0, 1], n_records, p=[0.6, 0.4])\n",
    "}\n",
    "\n",
    "df = pd.DataFrame(data)\n",
    "\n",
    "print(f\"Generated {len(df)} listings\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Exploration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check data info\n",
    "print(f\"Shape: {df.shape}\")\n",
    "print(f\"\\nData types:\\n{df.dtypes}\")\n",
    "print(f\"\\nMissing values:\\n{df.isnull().sum()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Summary statistics\n",
    "df.describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distribution of room types\n",
    "plt.figure(figsize=(10, 6))\n",
    "df['room_type'].value_counts().plot(kind='bar', color='steelblue')\n",
    "plt.title('Distribution of Room Types')\n",
    "plt.xlabel('Room Type')\n",
    "plt.ylabel('Count')\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Price distribution\n",
    "fig, axes = plt.subplots(1, 2, figsize=(12, 5))\n",
    "\n",
    "axes[0].hist(df['price'], bins=50, color='coral', edgecolor='black')\n",
    "axes[0].set_title('Price Distribution')\n",
    "axes[0].set_xlabel('Price ($)')\n",
    "axes[0].set_ylabel('Frequency')\n",
    "\n",
    "df.boxplot(column='price', by='room_type', ax=axes[1])\n",
    "axes[1].set_title('Price by Room Type')\n",
    "axes[1].set_xlabel('Room Type')\n",
    "axes[1].set_ylabel('Price ($)')\n",
    "plt.suptitle('')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Remove duplicates\n",
    "print(f\"Before: {len(df)} rows\")\n",
    "df = df.drop_duplicates(subset=['listing_id'])\n",
    "print(f\"After: {len(df)} rows\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Handle outliers using IQR method\n",
    "q1 = df['price'].quantile(0.25)\n",
    "q3 = df['price'].quantile(0.75)\n",
    "iqr = q3 - q1\n",
    "\n",
    "lower_bound = q1 - 1.5 * iqr\n",
    "upper_bound = q3 + 1.5 * iqr\n",
    "\n",
    "print(f\"Price outlier bounds: [{lower_bound:.2f}, {upper_bound:.2f}]\")\n",
    "\n",
    "df_clean = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)].copy()\n",
    "print(f\"Removed {len(df) - len(df_clean)} outliers\")\n",
    "print(f\"Final dataset: {len(df_clean)} rows\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature engineering\n",
    "df_clean['price_tier'] = pd.cut(df_clean['price'], \n",
    "                                  bins=[0, 100, 200, 500, 1000], \n",
    "                                  labels=['Budget', 'Mid-range', 'Premium', 'Luxury'])\n",
    "\n",
    "df_clean['has_reviews'] = (df_clean['number_of_reviews'] > 0).astype(int)\n",
    "\n",
    "print(\"\\nPrice tier distribution:\")\n",
    "print(df_clean['price_tier'].value_counts())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. A/B Test Group Assignment\n",
    "\n",
    "Randomly assign listings to control (A) or treatment (B) groups."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Random 50/50 assignment\n",
    "rng_assign = np.random.default_rng(42)\n",
    "df_clean['ab_group'] = rng_assign.choice(['A', 'B'], size=len(df_clean), p=[0.5, 0.5])\n",
    "\n",
    "group_counts = df_clean['ab_group'].value_counts()\n",
    "print(f\"Group A (Control): {group_counts['A']} listings\")\n",
    "print(f\"Group B (Treatment): {group_counts['B']} listings\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check balance across groups\n",
    "print(\"Average price by group:\")\n",
    "print(df_clean.groupby('ab_group')['price'].mean())\n",
    "\n",
    "print(\"\\nRoom type distribution by group:\")\n",
    "print(pd.crosstab(df_clean['ab_group'], df_clean['room_type'], normalize='index'))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Simulate Treatment Effect\n",
    "\n",
    "Apply simulated treatment effect to group B:\n",
    "- 10% increase in booking rate\n",
    "- 5% increase in price"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate baseline booking rates\n",
    "rng_booking = np.random.default_rng(42)\n",
    "df_clean['booking_rate'] = rng_booking.uniform(0.1, 0.4, len(df_clean))\n",
    "\n",
    "# Apply treatment effect to group B\n",
    "treatment_mask = df_clean['ab_group'] == 'B'\n",
    "df_clean.loc[treatment_mask, 'booking_rate'] *= 1.10\n",
    "df_clean.loc[treatment_mask, 'price'] *= 1.05\n",
    "\n",
    "# Calculate bookings and revenue\n",
    "df_clean['bookings'] = (df_clean['booking_rate'] * df_clean['availability_365']).astype(int)\n",
    "df_clean['revenue'] = df_clean['bookings'] * df_clean['price']\n",
    "\n",
    "print(\"Metrics by group:\")\n",
    "print(df_clean.groupby('ab_group')[['booking_rate', 'price', 'bookings', 'revenue']].mean())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Statistical Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# T-test for booking rate\n",
    "group_a_bookings = df_clean[df_clean['ab_group'] == 'A']['booking_rate']\n",
    "group_b_bookings = df_clean[df_clean['ab_group'] == 'B']['booking_rate']\n",
    "\n",
    "t_stat, p_value = stats.ttest_ind(group_a_bookings, group_b_bookings)\n",
    "\n",
    "print(\"T-test results for booking rate:\")\n",
    "print(f\"T-statistic: {t_stat:.4f}\")\n",
    "print(f\"P-value: {p_value:.4f}\")\n",
    "print(f\"Significant at 95% confidence: {p_value < 0.05}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate lift\n",
    "mean_a = group_a_bookings.mean()\n",
    "mean_b = group_b_bookings.mean()\n",
    "lift = ((mean_b - mean_a) / mean_a) * 100\n",
    "\n",
    "print(f\"\\nGroup A mean booking rate: {mean_a:.4f}\")\n",
    "print(f\"Group B mean booking rate: {mean_b:.4f}\")\n",
    "print(f\"Lift: {lift:.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare metrics\n",
    "metrics = df_clean.groupby('ab_group')[['booking_rate', 'revenue']].mean()\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 6))\n",
    "\n",
    "# Booking rate comparison\n",
    "metrics['booking_rate'].plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c'])\n",
    "axes[0].set_title('Average Booking Rate by Group', fontsize=14, fontweight='bold')\n",
    "axes[0].set_xlabel('A/B Group')\n",
    "axes[0].set_ylabel('Booking Rate')\n",
    "axes[0].set_xticklabels(['Group A (Control)', 'Group B (Treatment)'], rotation=0)\n",
    "\n",
    "# Revenue comparison\n",
    "metrics['revenue'].plot(kind='bar', ax=axes[1], color=['#3498db', '#e74c3c'])\n",
    "axes[1].set_title('Average Revenue by Group', fontsize=14, fontweight='bold')\n",
    "axes[1].set_xlabel('A/B Group')\n",
    "axes[1].set_ylabel('Revenue ($)')\n",
    "axes[1].set_xticklabels(['Group A (Control)', 'Group B (Treatment)'], rotation=0)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distribution comparison\n",
    "fig, axes = plt.subplots(1, 2, figsize=(12, 6))\n",
    "\n",
    "axes[0].hist(df_clean[df_clean['ab_group'] == 'A']['booking_rate'], bins=30, alpha=0.7, label='Group A', color='blue')\n",
    "axes[0].hist(df_clean[df_clean['ab_group'] == 'B']['booking_rate'], bins=30, alpha=0.7, label='Group B', color='red')\n",
    "axes[0].set_xlabel('Booking Rate')\n",
    "axes[0].set_ylabel('Frequency')\n",
    "axes[0].set_title('Booking Rate Distribution')\n",
    "axes[0].legend()\n",
    "\n",
    "axes[1].hist(df_clean[df_clean['ab_group'] == 'A']['revenue'], bins=30, alpha=0.7, label='Group A', color='blue')\n",
    "axes[1].hist(df_clean[df_clean['ab_group'] == 'B']['revenue'], bins=30, alpha=0.7, label='Group B', color='red')\n",
    "axes[1].set_xlabel('Revenue ($)')\n",
    "axes[1].set_ylabel('Frequency')\n",
    "axes[1].set_title('Revenue Distribution')\n",
    "axes[1].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Save Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save cleaned data\n",
    "output_path = '../data/airbnb_clean.csv'\n",
    "df_clean.to_csv(output_path, index=False)\n",
    "print(f\"Saved cleaned data to {output_path}\")\n",
    "print(f\"Total records: {len(df_clean)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save to SQLite database\n",
    "db_path = '../data/airbnb_ab_test.db'\n",
    "conn = sqlite3.connect(db_path)\n",
    "df_clean.to_sql('listings', conn, if_exists='replace', index=False)\n",
    "conn.close()\n",
    "print(f\"Saved data to database: {db_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Summary Statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"A/B Test Summary\")\n",
    "print(\"=\"*50)\n",
    "print(f\"Total listings: {len(df_clean)}\")\n",
    "print(f\"Group A: {len(df_clean[df_clean['ab_group'] == 'A'])}\")\n",
    "print(f\"Group B: {len(df_clean[df_clean['ab_group'] == 'B'])}\")\n",
    "print(f\"\\nAverage booking rate lift: {lift:.2f}%\")\n",
    "print(f\"Statistical significance: {p_value < 0.05}\")\n",
    "print(f\"\\nNext step: Run SQL analysis using queries in sql/ folder\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}